In [1]:
from smartpool_config import *

spark = create_spark("smartpool-gold")

print("Spark OK:", spark.version)
print("BASE:", BASE)
print("BRONZE:", BRONZE)
print("SILVER:", SILVER)
print("GOLD:", GOLD)
print("STATE:", STATE)
print("JDBC:", JDBC_URL)


:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/jovyan/.ivy2.5.2/cache
The jars for the packages stored in: /home/jovyan/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6435f54f-1f84-4579-bad1-4175e91d1be6;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.13/4.0.0/delta-spark_2.13-4.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.13;4.0.0!delta-spark_2.13.jar (7060ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/4.0.0/delta-storage-4.0.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;4.0.0!delta-storage.jar (443ms)
downloading https://repo1.maven.org/maven2/org/antlr/ant

Spark OK: 4.0.1
BASE: s3a://spark/medallion
BRONZE: s3a://spark/medallion/bronze
SILVER: s3a://spark/medallion/silver
GOLD: s3a://spark/medallion/gold
STATE: s3a://spark/medallion/_state
JDBC: jdbc:sqlserver://sqlserver:1433;databaseName=smartpool;encrypt=true;trustServerCertificate=true;


In [2]:
POOLS_SILVER  = f"{SILVER}/pools_dim"
EVENTS_SILVER = f"{SILVER}/maintenance_events"

GOLD_DAILY    = f"{GOLD}/pool_daily_metrics"
GOLD_LATEST   = f"{GOLD}/pool_latest_event"

print("POOLS_SILVER:", POOLS_SILVER)
print("EVENTS_SILVER:", EVENTS_SILVER)
print("GOLD_DAILY:", GOLD_DAILY)
print("GOLD_LATEST:", GOLD_LATEST)

POOLS_SILVER: s3a://spark/medallion/silver/pools_dim
EVENTS_SILVER: s3a://spark/medallion/silver/maintenance_events
GOLD_DAILY: s3a://spark/medallion/gold/pool_daily_metrics
GOLD_LATEST: s3a://spark/medallion/gold/pool_latest_event


In [3]:
pools_s  = spark.read.format("delta").load(POOLS_SILVER)
events_s = spark.read.format("delta").load(EVENTS_SILVER)

print("pools_s rows :", pools_s.count())
print("events_s rows:", events_s.count())

events_s.select("pool_id", "event_time", "event_date", "intervention_type", "product_amount", "updated_at").show(10, truncate=False)

26/01/25 20:46:53 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
26/01/25 20:46:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


pools_s rows : 6


events_s rows: 16


+-------+--------------------------+----------+-----------------+--------------+--------------------------+
|pool_id|event_time                |event_date|intervention_type|product_amount|updated_at                |
+-------+--------------------------+----------+-----------------+--------------+--------------------------+
|1      |2026-01-15 01:10:41.856293|2026-01-15|chlorine         |250.0         |2026-01-25 01:10:41.856293|
|2      |2026-01-17 01:10:41.856293|2026-01-17|chlorine         |200.0         |2026-01-25 01:10:41.856293|
|3      |2026-01-19 01:10:41.856293|2026-01-19|chlorine         |220.0         |2026-01-25 01:10:41.856293|
|2      |2026-01-24 01:10:41.856293|2026-01-24|ph_correction    |120.0         |2026-01-25 01:10:41.856293|
|5      |2026-01-24 01:10:41.856293|2026-01-24|ph_correction    |200.0         |2026-01-25 01:10:41.856293|
|1      |2026-01-18 01:10:41.856293|2026-01-18|ph_correction    |150.0         |2026-01-25 01:10:41.856293|
|5      |2026-01-18 01:10:41

In [4]:
daily = (
    events_s
    .groupBy("pool_id", "event_date")
    .agg(
        F.count("*").alias("num_events"),
        F.sum(F.when(F.col("product_amount").isNotNull(), F.col("product_amount")).otherwise(F.lit(0.0))).alias("total_product_amount"),
        F.sum(F.when(F.col("intervention_type") == "chlorine", 1).otherwise(0)).alias("n_chlorine"),
        F.sum(F.when(F.col("intervention_type") == "ph_correction", 1).otherwise(0)).alias("n_ph_correction"),
        F.sum(F.when(F.col("intervention_type") == "refill", 1).otherwise(0)).alias("n_refill"),
        F.sum(F.when(F.col("intervention_type") == "filter_backwash", 1).otherwise(0)).alias("n_filter_backwash"),
        F.max("event_time").alias("last_event_time"),
        F.current_timestamp().alias("calc_ts"),
    )
)

daily.orderBy(F.col("event_date").desc(), F.col("pool_id")).show(50, truncate=False)
print("daily rows:", daily.count())

+-------+----------+----------+--------------------+----------+---------------+--------+-----------------+--------------------------+--------------------------+
|pool_id|event_date|num_events|total_product_amount|n_chlorine|n_ph_correction|n_refill|n_filter_backwash|last_event_time           |calc_ts                   |
+-------+----------+----------+--------------------+----------+---------------+--------+-----------------+--------------------------+--------------------------+
|1      |2026-01-25|2         |270.0               |1         |1              |0       |0                |2026-01-25 19:09:21.820564|2026-01-25 20:47:10.418557|
|2      |2026-01-24|1         |120.0               |0         |1              |0       |0                |2026-01-24 01:10:41.856293|2026-01-25 20:47:10.418557|
|5      |2026-01-24|1         |200.0               |0         |1              |0       |0                |2026-01-24 01:10:41.856293|2026-01-25 20:47:10.418557|
|3      |2026-01-23|1         |0.0

In [5]:
daily_enriched = (
    daily.alias("d")
    .join(
        pools_s.select("pool_id", "pool_name", "location", "owner_type", "volume_liters", "is_heated").alias("p"),
        on="pool_id",
        how="left"
    )
    .select(
        "pool_id", "pool_name", "location", "owner_type", "volume_liters", "is_heated",
        "event_date", "num_events", "total_product_amount",
        "n_chlorine", "n_ph_correction", "n_refill", "n_filter_backwash",
        "last_event_time", "calc_ts"
    )
)

daily_enriched.orderBy(F.col("event_date").desc(), F.col("pool_id")).show(50, truncate=False)
print("daily_enriched rows:", daily_enriched.count())


+-------+-------------------------------+---------------------+-------------+-------------+---------+----------+----------+--------------------+----------+---------------+--------+-----------------+--------------------------+--------------------------+
|pool_id|pool_name                      |location             |owner_type   |volume_liters|is_heated|event_date|num_events|total_product_amount|n_chlorine|n_ph_correction|n_refill|n_filter_backwash|last_event_time           |calc_ts                   |
+-------+-------------------------------+---------------------+-------------+-------------+---------+----------+----------+--------------------+----------+---------------+--------+-----------------+--------------------------+--------------------------+
|1      |Piscina Casa Pueblo            |Valdeganga (Albacete)|private      |40000        |false    |2026-01-25|2         |270.0               |1         |1              |0       |0                |2026-01-25 19:09:21.820564|2026-01-25 20:47

In [6]:
(daily_enriched.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy("event_date")
    .save(GOLD_DAILY)
)

print("GOLD daily OK ->", GOLD_DAILY)

GOLD daily OK -> s3a://spark/medallion/gold/pool_daily_metrics


In [7]:
w = Window.partitionBy("pool_id").orderBy(F.col("event_time").desc(), F.col("updated_at").desc(), F.col("id").desc())

latest_event = (
    events_s
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .drop("rn")
    .withColumn("calc_ts", F.current_timestamp())
)

latest_event.orderBy("pool_id").show(50, truncate=False)
print("latest_event rows:", latest_event.count())

+-------+----+--------------------------+-----------------+------------+--------------+---------------------------------------------------+--------------------------+----------+--------------------------+
|pool_id|id  |event_time                |intervention_type|product_type|product_amount|notes                                              |updated_at                |event_date|calc_ts                   |
+-------+----+--------------------------+-----------------+------------+--------------+---------------------------------------------------+--------------------------+----------+--------------------------+
|1      |1003|2026-01-25 19:09:21.820564|ph_correction    |minus       |90.0          |Evento incremental de prueba 2                     |2026-01-25 19:07:21.820564|2026-01-25|2026-01-25 20:47:18.512766|
|2      |6   |2026-01-24 01:10:41.856293|ph_correction    |plus        |120.0         |Corrección de pH por agua demasiado ácida          |2026-01-25 01:10:41.856293|2026-01-24|202

In [8]:
(latest_event.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(GOLD_LATEST)
)

print("GOLD latest OK ->", GOLD_LATEST)


GOLD latest OK -> s3a://spark/medallion/gold/pool_latest_event


In [9]:
daily_g = spark.read.format("delta").load(GOLD_DAILY)
latest_g = spark.read.format("delta").load(GOLD_LATEST)

print("GOLD daily rows :", daily_g.count())
print("GOLD latest rows:", latest_g.count())

# Unicidad esperada
dups_daily = daily_g.groupBy("pool_id","event_date").count().filter("count > 1").count()
dups_latest = latest_g.groupBy("pool_id").count().filter("count > 1").count()
print("[QA] dups daily (pool_id,event_date):", dups_daily)
print("[QA] dups latest (pool_id):", dups_latest)

# Referencial (daily debe tener pool_id existente)
missing_pool_daily = (daily_g.select("pool_id").distinct()
    .join(pools_s.select("pool_id").distinct(), on="pool_id", how="left_anti")
    .count()
)
print("[QA] daily pool_id sin pools_dim:", missing_pool_daily)

print("Delta history GOLD_DAILY:")
display(DeltaTable.forPath(spark, GOLD_DAILY).history(10))

print("Delta history GOLD_LATEST:")
display(DeltaTable.forPath(spark, GOLD_LATEST).history(10))

print("04_gold OK")


GOLD daily rows : 15
GOLD latest rows: 5
[QA] dups daily (pool_id,event_date): 0
[QA] dups latest (pool_id): 0
[QA] daily pool_id sin pools_dim: 0
Delta history GOLD_DAILY:


DataFrame[version: bigint, timestamp: timestamp, userId: string, userName: string, operation: string, operationParameters: map<string,string>, job: struct<jobId:string,jobName:string,jobRunId:string,runId:string,jobOwnerId:string,triggerType:string>, notebook: struct<notebookId:string>, clusterId: string, readVersion: bigint, isolationLevel: string, isBlindAppend: boolean, operationMetrics: map<string,string>, userMetadata: string, engineInfo: string]

Delta history GOLD_LATEST:


DataFrame[version: bigint, timestamp: timestamp, userId: string, userName: string, operation: string, operationParameters: map<string,string>, job: struct<jobId:string,jobName:string,jobRunId:string,runId:string,jobOwnerId:string,triggerType:string>, notebook: struct<notebookId:string>, clusterId: string, readVersion: bigint, isolationLevel: string, isBlindAppend: boolean, operationMetrics: map<string,string>, userMetadata: string, engineInfo: string]

04_gold OK


In [10]:
spark.stop()